In [1]:
# 01 - Setup: device, imports, seeds, paths, model

import os
from pathlib import Path
import random
import re
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional

# ---- Single toggle: "cpu" or "gpu"
RUN_DEVICE = "cpu"   # change to "gpu" if you have CUDA

os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
if RUN_DEVICE.lower() == "gpu" and torch.cuda.is_available():
    SELECT_DEVICE = "cuda"
else:
    SELECT_DEVICE = "cpu"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

device = torch.device(SELECT_DEVICE)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
if device.type == "cpu":
    torch.set_num_threads(max(1, os.cpu_count() // 2))

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

from sklearn.metrics import f1_score, classification_report, confusion_matrix

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)

# Paths
EXCEL_PATH = Path("Punjabi Dataset.xlsx")
OUT_DIR = Path("outputs_punjabi_metaphor")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Model
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
MAX_NEW_TOKENS = 1
DO_SAMPLE = False

print(f"Using device: {device.type.upper()} | Model: {MODEL_NAME}")

Using device: CPU | Model: meta-llama/Llama-3.2-3B-Instruct


In [2]:
# 02 - Load and inspect data

# Load the Punjabi metaphor dataset
df = pd.read_excel(EXCEL_PATH)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst 3 rows:")
print(df.head(3))
print(f"\nData types:\n{df.dtypes}")

# For this dataset, we'll treat "Metaphor_Used" as the key metaphor expression
# We need to assign binary labels (0=literal, 1=metaphorical)
# Since we don't have ground truth labels, let's create a binary target based on sentence analysis

# Create a simple feature: if the target sentence explicitly says "रूपक" (metaphor in Hindi)
# or similar indicators, mark as 1 (metaphorical), else 0 (literal)

df['Label'] = 0  # Default to literal
# Mark as metaphorical if sentence mentions it's a metaphor
df.loc[df['Target_Sentence'].str.contains('रूपक|ਰੂਪਕ|metaphor', case=False, na=False), 'Label'] = 1

print(f"\nLabel distribution:")
print(df['Label'].value_counts())
print(f"\nData ready for classification!")

Dataset shape: (50, 5)
Columns: ['ID', 'Metaphor_Used', 'Prev_Sentence', 'Target_Sentence', 'Final_Sentence']

First 3 rows:
   ID         Metaphor_Used                    Prev_Sentence  \
0   1  ਕਾਰ ਵਿਚ ਅੱਗ ਲੱਗ ਜਾਣਾ      ਚਾਹ ਕਾਫ਼ੀ ਠੰਢੀ ਹੋ ਚੁੱਕੀ ਸੀ।   
1   2          ਕੂਏਂ ਦਾ ਮੱਛੀ  ਰਾਤ ਨੂੰ ਕੁੱਤਾ ਬਹੁਤ ਭੌਂਕਦਾ ਰਿਹਾ।   
2   3          ਅੱਗ ਲੱਗ ਜਾਣਾ      ਚਾਹ ਕਾਫ਼ੀ ਠੰਢੀ ਹੋ ਚੁੱਕੀ ਸੀ।   

                                     Target_Sentence  \
0  ਕਾਰ ਵਿਚ ਅੱਗ ਲੱਗ ਜਾਣਾ ਸੱਚਮੁੱਚ ਹੋਇਆ ਸੀ, ਜਿਵੇਂ ਕਿ...   
1  ਕੂਏਂ ਦਾ ਮੱਛੀ ਸਿਰਫ਼ ਰੂਪਕ ਸੀ, ਪਰ ਇਸਦਾ ਮਤਲਬ ਕਾਫ਼ੀ...   
2  ਅੱਗ ਲੱਗ ਜਾਣਾ ਸਿਰਫ਼ ਰੂਪਕ ਸੀ, ਪਰ ਇਸਦਾ ਮਤਲਬ ਕਾਫ਼ੀ...   

                    Final_Sentence  
0   ਫਿਰ ਉਸਨੇ ਘਰ ਜਾਣ ਦਾ ਫੈਸਲਾ ਕੀਤਾ।  
1   ਫਿਰ ਸਾਰੇ ਆਪਣੇ ਕੰਮ ਵਿੱਚ ਲੱਗ ਗਏ।  
2  ਉਸ ਤੋਂ ਬਾਅਦ ਮਾਹੌਲ ਸ਼ਾਂਤ ਹੋ ਗਿਆ।  

Data types:
ID                  int64
Metaphor_Used      object
Prev_Sentence      object
Target_Sentence    object
Final_Sentence     object
dtype: object

Label distribution:
Label
0    25
1    25
Name: count, dtype: int64

Data ready for cla

In [3]:
# 03 - Load model and tokenizer

import gc
from huggingface_hub import login as hf_login
from getpass import getpass

# Optional HF login for gated models
hf_token = os.getenv("HF_TOKEN", "").strip()
if not hf_token:
    try:
        hf_token = getpass("Enter HuggingFace token (or press Enter to skip): ").strip()
    except:
        pass

if hf_token:
    try:
        hf_login(token=hf_token)
        print("✓ Logged in to Hugging Face")
    except Exception as e:
        print(f"Warning: HF login failed: {e}")

_token_arg = {"token": hf_token} if hf_token else {}

# Load tokenizer
print(f"Loading tokenizer for {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
    trust_remote_code=True,
    **_token_arg
)

# Load model
print(f"Loading model...")
if device.type == "cuda":
    dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=dtype,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        **_token_arg
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32,
        device_map={"": "cpu"},
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        **_token_arg
    )

model.eval()

# Ensure pad token
if tokenizer.pad_token_id is None:
    if tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.add_special_tokens({"pad_token": "[PAD]"})
        model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

print(f"✓ Model and tokenizer loaded!")

✓ Logged in to Hugging Face
Loading tokenizer for meta-llama/Llama-3.2-3B-Instruct...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model and tokenizer loaded!


In [4]:
# 04 - Prompt templates

INSTR_ZS = (
    "Is the Punjabi metaphor/idiom used metaphorically (figuratively) in the sentence below? "
    "Respond with only '1' for metaphorical/figurative or '0' for literal."
)

def make_prompt(metaphor: str, prev_sent: str, target_sent: str, next_sent: str) -> str:
    return (
        f"{INSTR_ZS}\n\n"
        f"Metaphor: {metaphor}\n\n"
        f"Previous Sentence: {prev_sent}\n"
        f"Target Sentence: {target_sent}\n"
        f"Next Sentence: {next_sent}\n\n"
        f"Answer:"
    )

# Test prompt
test_row = df.iloc[0]
test_prompt = make_prompt(
    test_row['Metaphor_Used'],
    test_row['Prev_Sentence'],
    test_row['Target_Sentence'],
    test_row['Final_Sentence']
)
print("Sample prompt:")
print(test_prompt[:300] + "...")

Sample prompt:
Is the Punjabi metaphor/idiom used metaphorically (figuratively) in the sentence below? Respond with only '1' for metaphorical/figurative or '0' for literal.

Metaphor: ਕਾਰ ਵਿਚ ਅੱਗ ਲੱਗ ਜਾਣਾ

Previous Sentence: ਚਾਹ ਕਾਫ਼ੀ ਠੰਢੀ ਹੋ ਚੁੱਕੀ ਸੀ।
Target Sentence: ਕਾਰ ਵਿਚ ਅੱਗ ਲੱਗ ਜਾਣਾ ਸੱਚਮੁੱਚ ਹੋਇਆ ਸੀ, ਜਿਵੇਂ ਕ...


In [5]:
# 05 - Classification function

# Cache token IDs for "0" and "1"
_id0 = None
_id1 = None

def _candidate_token_id_for_digit(d: str) -> Optional[int]:
    ids = tokenizer.encode(d, add_special_tokens=False)
    if len(ids) == 1: return ids[0]
    ids = tokenizer.encode(" " + d, add_special_tokens=False)
    if len(ids) == 1: return ids[0]
    ids = tokenizer.encode(d + "\n", add_special_tokens=False)
    if len(ids) == 1: return ids[0]
    return None

def _init_digit_ids():
    global _id0, _id1
    if _id0 is None: _id0 = _candidate_token_id_for_digit("0")
    if _id1 is None: _id1 = _candidate_token_id_for_digit("1")

_init_digit_ids()
print(f"Token ID for '0': {_id0}")
print(f"Token ID for '1': {_id1}")

@torch.no_grad()
def classify_prompt(prompt: str) -> int:
    """Classify a single prompt by comparing logits of '0' and '1' tokens"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    logits = model(**inputs).logits
    next_logits = logits[0, -1, :]
    
    if _id0 is not None and _id1 is not None:
        logit0 = next_logits[_id0].item()
        logit1 = next_logits[_id1].item()
        return 1 if logit1 >= logit0 else 0
    
    # Fallback: generate 1 token and check content
    gen = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        do_sample=DO_SAMPLE,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    text = tokenizer.decode(gen[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    return 1 if "1" in text else 0

print("Classification function ready!")

Token ID for '0': 15
Token ID for '1': 16
Classification function ready!


In [6]:
# 06 - Batch classification with progress

from tqdm.auto import tqdm

def classify_batch(df_batch: pd.DataFrame) -> List[int]:
    """Classify a batch of samples"""
    predictions = []
    for _, row in tqdm(df_batch.iterrows(), total=len(df_batch), desc="Classifying"):
        prompt = make_prompt(
            row['Metaphor_Used'],
            row['Prev_Sentence'],
            row['Target_Sentence'],
            row['Final_Sentence']
        )
        pred = classify_prompt(prompt)
        predictions.append(pred)
    return predictions

# Run classification on the full dataset
print(f"Starting classification on {len(df)} samples...")
y_pred = classify_batch(df)
y_true = df['Label'].tolist()

print(f"\n✓ Classification complete!")
print(f"Predictions: {pd.Series(y_pred).value_counts().to_dict()}")

Starting classification on 50 samples...


Classifying:   0%|          | 0/50 [00:00<?, ?it/s]


✓ Classification complete!
Predictions: {1: 50}


In [7]:
# 07 - Evaluation

f1 = f1_score(y_true, y_pred, average="binary")
print(f"\n=== Classification Results ===")
print(f"Macro F1 Score: {f1:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Literal', 'Metaphorical'], digits=4))
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Save results
results_df = df.copy()
results_df['Prediction'] = y_pred
results_df['Correct'] = (results_df['Label'] == results_df['Prediction']).astype(int)

results_path = OUT_DIR / "punjabi_metaphor_predictions.csv"
results_df.to_csv(results_path, index=False)
print(f"\n✓ Results saved to {results_path}")

# Summary
print(f"\nAccuracy: {results_df['Correct'].mean():.4f}")


=== Classification Results ===
Macro F1 Score: 0.6667

Classification Report:
              precision    recall  f1-score   support

     Literal     0.0000    0.0000    0.0000        25
Metaphorical     0.5000    1.0000    0.6667        25

    accuracy                         0.5000        50
   macro avg     0.2500    0.5000    0.3333        50
weighted avg     0.2500    0.5000    0.3333        50


Confusion Matrix:
[[ 0 25]
 [ 0 25]]


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



✓ Results saved to outputs_punjabi_metaphor/punjabi_metaphor_predictions.csv

Accuracy: 0.5000
